# Algo testing

## Dependencies

### Imports

In [2]:
import requests
import json
import plotly
import time
import datetime

### Init

In [3]:
plotly.offline.init_notebook_mode()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


### Helpers

In [4]:
def http_get(url):
    r = requests.get(url)
    return json.loads(r.text)

def parse_unix_date(posix_time):
    return datetime.datetime.utcfromtimestamp(posix_time).strftime('%Y-%m-%dT%H:%M:%SZ')

## Fetching charts

### Poloniex

#### BTC/USDT

Fetching BTC/USDT from the poloniex API.

In [28]:
chart_data_poloniex_btcusdt = http_get('https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start=1305699200&end=9999999999&period=300')
print(chart_data_poloniex_btcusdt[0])
print("Records: " + str(len(chart_data_poloniex_btcusdt)))

{'close': 225, 'quoteVolume': 0.00444444, 'date': 1424373000, 'open': 0.33, 'volume': 0.999999, 'weightedAverage': 225, 'low': 225, 'high': 0.33}
Records: 277922


## BTC/USDT Poloniex

In [29]:
length = 100
extra = 20
filtered = chart_data_poloniex_btcusdt[len(chart_data_poloniex_btcusdt)-(length+extra)-1:len(chart_data_poloniex_btcusdt)-1]

ma_size = 7
sigma = 2
for index, f in enumerate(filtered[ma_size:]):
    ma = sum([i['close'] for i in filtered[index:index+ma_size]]) / ma_size
    stdev = (sum( [(i['close']-ma)**2 for i in filtered[index:index+ma_size]]) / (ma_size-1))**(1/2)
    filtered[index+ma_size]['ma'] = ma
    filtered[index+ma_size]['stdev'] = stdev
    filtered[index+ma_size]['bbup'] = ma + (stdev*sigma)
    filtered[index+ma_size]['bbdown'] = ma - (stdev*sigma)
    
def MovingAverage(periods, data):
    for index, f in enumerate(data[periods:]):
        ma = sum([i['close'] for i in data[index:index+periods]]) / periods
        data[index+periods]['ma_' + str(periods)] = ma

MovingAverage(8, filtered)
MovingAverage(13, filtered)
MovingAverage(21, filtered)
MovingAverage(55, filtered)

x, y, ma8, ma13, ma21, ma55 = zip(*[(parse_unix_date(f['date']), f['close'], f['ma_8'], f['ma_13'], f['ma_21'], f['ma_55']) for f in filtered[55:]])

trace = plotly.graph_objs.Scatter(
    x = x,
    y = y,
)
trace1 = plotly.graph_objs.Scatter(
    x = x,
    y = ma8,
    line = dict(
        color = ('rgb(225, 0, 0)'),
        width = 1,)
)
trace2 = plotly.graph_objs.Scatter(
    x = x,
    y = ma13,
    line = dict(
        color = ('rgb(185, 0, 0)'),
        width = 1,)
)
trace3 = plotly.graph_objs.Scatter(
    x = x,
    y = ma21,
    line = dict(
        color = ('rgb(145, 0, 0)'),
        width = 1,)
)
trace4 = plotly.graph_objs.Scatter(
    x = x,
    y = ma55,
    line = dict(
        color = ('rgb(200, 200, 200)'),
        width = 1,)
)
layout = dict(
    xaxis = dict(title=""),
    yaxis = dict(title="")
)
data = [trace,trace1,trace2,trace3,trace4]
fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig)

In [31]:
for index, f in enumerate(filtered):
    if index == 0:
        filtered[index]["roc"] = 0
        continue
    filtered[index]["roc"] = filtered[index]["close"] - filtered[index-1]["close"]
    
x, y = zip(*[(parse_unix_date(f["date"]), f["roc"]) for f in filtered[55:]])

trace = plotly.graph_objs.Scatter(
    x = x,
    y = y
)

layout = dict(
    xaxis = dict(title=""),
    yaxis = dict(title="")
)
data = [trace]
fig = dict(data=data, layout=layout)
plotly.offline.iplot(fig)